In [2]:
import pandas as pd
import time
from flask import current_app
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
import math

In [3]:
# Add working directory
import os
os.chdir("C:/Users/Swati Arora/Data Science using Python/Movie Recommender System- Pybots/Data")

In [4]:
#Reading the files 

In [5]:
#genre_matrix is a preprocessed file
genre_matrix=pd.read_csv("genre_matrix.csv",sep=',')
Ratings=pd.read_csv("ratings.dat",sep='::', encoding='latin-1',names=['UserID','MovieID','Rating','Timestamp'])
Movies=pd.read_csv('movies.dat',sep='::',encoding='ANSI',names=['MovieID','Title','Genres'])

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


In [6]:
Movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
Movies.shape

(3883, 3)

In [8]:
Ratings.shape

(1000209, 4)

In [9]:
genre_matrix.shape

(3883, 20)

In [10]:
genre_matrix.head()

,Unnamed: 0,MovieID,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,3,4,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,4,5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
genre_matrix.drop('Unnamed: 0',axis=1,inplace=True)

In [12]:
#Movies=Movies.rename(columns = {'movieId':'MovieID'})

In [13]:
#Ratings=Ratings.rename(columns = {'movieId':'MovieID'})

In [14]:
Ratings.head()


,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [15]:
#len(genre_matrix.count)

In [16]:
(Ratings['MovieID']).unique()

array([1193,  661,  914, ..., 2845, 3607, 2909], dtype=int64)

# Creation of Item profile based in the genre

In [17]:
binaryRatings = Ratings

In [18]:
genre_matrix.head()

,MovieID,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,4,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
genre_matrix.head()
genre_matrix.shape

(3883, 19)

In [20]:
items=genre_matrix
items.shape

(3883, 19)

In [21]:
items.head()

,MovieID,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,4,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
#Taking the files for which rating is available 
items2 = items[items['MovieID'].isin(binaryRatings['MovieID'].tolist())]
items2.shape

(3706, 19)

In [23]:
items2_sort =items2.sort_values(by='MovieID',ascending=True)

In [24]:
items2_sort.head()


,MovieID,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,4,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
itemprof =items2_sort

In [26]:
np.unique(itemprof['MovieID'])

array([   1,    2,    3, ..., 3950, 3951, 3952], dtype=int64)

In [27]:
# Creation of binary Rating 

In [28]:
binaryRatings.drop('Timestamp',axis=1)
type(binaryRatings)
binaryRatings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [29]:
#converting teh rating to 1 and -1 if the rating is above 3 that means user liked the ovie otherwise user did not like 
for i in range(0, len(binaryRatings)):
    if(binaryRatings.iat[i,2]>3):
       binaryRatings.iloc[i,2]=1
    else :
       binaryRatings.iloc[i,2]=-1

In [30]:
#convert the data from long to wide format 
ratmat=binaryRatings.pivot(index='UserID',columns='MovieID',values='Rating').fillna(0)




In [31]:
len(ratmat.index)

6040

In [32]:
ratmat1=ratmat.to_dense().as_matrix()
ratmat1.shape

(6040, 3706)

In [33]:
itemprof.head()

,MovieID,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,4,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [34]:
itemprof.set_index('MovieID',inplace=True)

In [35]:
pd.DataFrame(ratmat1).head()

,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Creation of User Profile by dot product

In [36]:
from scipy import linalg,dot
userprof = dot(ratmat1,itemprof)/linalg.norm(ratmat1)/linalg.norm(itemprof)
userprof

array([[  3.81206592e-05,   3.81206592e-05,   1.27068864e-04, ...,
          1.27068864e-05,   2.54137728e-05,   0.00000000e+00],
       [ -5.08275456e-05,   3.81206592e-05,   0.00000000e+00, ...,
         -1.27068864e-05,   1.27068864e-05,   3.81206592e-05],
       [  1.65189523e-04,   2.16017069e-04,   1.27068864e-05, ...,
          1.27068864e-05,   2.54137728e-05,   7.62413184e-05],
       ..., 
       [  0.00000000e+00,   1.27068864e-05,   1.27068864e-05, ...,
          0.00000000e+00,   2.54137728e-05,   0.00000000e+00],
       [  5.08275456e-05,   7.62413184e-05,   3.81206592e-05, ...,
          1.52482637e-04,   8.89482048e-05,   2.54137728e-05],
       [ -1.52482637e-04,  -1.27068864e-04,   0.00000000e+00, ...,
          3.17672160e-04,  -1.27068864e-05,   2.54137728e-05]])

In [37]:
userprof_df=pd.DataFrame(userprof)
userprof_df.shape

(6040, 18)

In [38]:
for i in range(0, userprof_df.shape[0]):
    for j in range(0,userprof_df.shape[1]):
        if(userprof_df.iat[i,j]>0):
           userprof_df.iloc[i,j]=1
        else :
           userprof_df.iloc[i,j]=0
    

# Jaccard similarity 

In [39]:
import itertools
import pandas as pd
import sklearn 
from sklearn.metrics.pairwise import pairwise_distances


In [40]:
# Method to compute Jaccard similarity index between two sets
def compute_jaccard(user1_vals, user2_vals):
    intersection = user1_vals.intersection(user2_vals)
    union = user1_vals.union(user2_vals)
    jaccard = len(intersection)/float(len(union))
    return jaccard

In [41]:
#Calculation of the similarity 
from sklearn.metrics.pairwise import pairwise_distances
similarity_jaccard= (1- pairwise_distances(userprof_df,itemprof,metric="hamming"))

In [42]:
similarity_jaccard_df = pd.DataFrame(similarity_jaccard)

In [43]:
similarity_jaccard_df_transpose=similarity_jaccard_df.transpose()
similarity_jaccard_df_transpose.shape

(3706, 6040)

In [44]:
similarity_jaccard_df_transpose.set_index(itemprof.index,inplace=True)

In [45]:
similarity_jaccard_df_transpose.head()

,0,1,2,3,4,5,6,7,8,9,...,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039
MovieID,,,,,,,,,,,,,,,,,,,,,
1,0.500000,0.555556,0.444444,0.388889,0.666667,0.611111,0.333333,0.333333,0.388889,0.333333,...,0.388889,0.277778,0.388889,0.388889,0.777778,0.444444,0.222222,0.666667,0.277778,0.333333
2,0.500000,0.555556,0.444444,0.611111,0.555556,0.500000,0.333333,0.222222,0.277778,0.333333,...,0.500000,0.277778,0.500000,0.388889,0.777778,0.333333,0.222222,0.555556,0.166667,0.333333
3,0.333333,0.722222,0.388889,0.333333,0.500000,0.555556,0.500000,0.388889,0.333333,0.277778,...,0.333333,0.222222,0.444444,0.444444,0.833333,0.388889,0.277778,0.722222,0.222222,0.388889
4,0.444444,0.722222,0.388889,0.444444,0.500000,0.555556,0.500000,0.388889,0.333333,0.277778,...,0.333333,0.222222,0.444444,0.555556,0.833333,0.500000,0.277778,0.722222,0.222222,0.388889
5,0.388889,0.666667,0.333333,0.388889,0.555556,0.500000,0.444444,0.333333,0.277778,0.222222,...,0.277778,0.166667,0.388889,0.500000,0.888889,0.444444,0.222222,0.666667,0.166667,0.333333


In [46]:
similarity_jaccard_df_transpose['MovieID'] = similarity_jaccard_df_transpose.index

In [47]:
similarity_jaccard_df_transpose.head()

,0,1,2,3,4,5,6,7,8,9,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,MovieID
MovieID,,,,,,,,,,,,,,,,,,,,,
1,0.500000,0.555556,0.444444,0.388889,0.666667,0.611111,0.333333,0.333333,0.388889,0.333333,...,0.277778,0.388889,0.388889,0.777778,0.444444,0.222222,0.666667,0.277778,0.333333,1
2,0.500000,0.555556,0.444444,0.611111,0.555556,0.500000,0.333333,0.222222,0.277778,0.333333,...,0.277778,0.500000,0.388889,0.777778,0.333333,0.222222,0.555556,0.166667,0.333333,2
3,0.333333,0.722222,0.388889,0.333333,0.500000,0.555556,0.500000,0.388889,0.333333,0.277778,...,0.222222,0.444444,0.444444,0.833333,0.388889,0.277778,0.722222,0.222222,0.388889,3
4,0.444444,0.722222,0.388889,0.444444,0.500000,0.555556,0.500000,0.388889,0.333333,0.277778,...,0.222222,0.444444,0.555556,0.833333,0.500000,0.277778,0.722222,0.222222,0.388889,4
5,0.388889,0.666667,0.333333,0.388889,0.555556,0.500000,0.444444,0.333333,0.277778,0.222222,...,0.166667,0.388889,0.500000,0.888889,0.444444,0.222222,0.666667,0.166667,0.333333,5


In [48]:
df =pd.merge(similarity_jaccard_df_transpose, Movies, how='left', on=['MovieID'])
df.shape


(3706, 6043)

In [49]:
df.drop('Genres',axis=1,inplace=True)

In [50]:
df.set_index(df['Title'],inplace=True)

In [51]:
df.drop('Title',axis=1,inplace=True)

# Recommending the movie to the user for which similarity is maximum

In [52]:
df.idxmax()

0                        Transformers: The Movie, The (1986)
1                                 Legends of the Fall (1994)
2                        Transformers: The Movie, The (1986)
3                  Star Wars: Episode IV - A New Hope (1977)
4                                            Hercules (1997)
5                                            Hercules (1997)
6                                 From Dusk Till Dawn (1996)
7                                 From Dusk Till Dawn (1996)
8                        Transformers: The Movie, The (1986)
9                        Transformers: The Movie, The (1986)
10                                    Dead Presidents (1995)
11                                        Hard Target (1993)
12                                        Richard III (1995)
13            Things to Do in Denver when You're Dead (1995)
14                                       Crimson Tide (1995)
15                               Prince of Egypt, The (1998)
16                      

# Recommending the top 5 movies to the particular user

In [53]:
#Recommending these 5 movies to the user id1
df[4441].nlargest(5)

Title
Lady and the Tramp (1955)     0.888889
Little Mermaid, The (1989)    0.888889
Pocahontas (1995)             0.833333
Goofy Movie, A (1995)         0.833333
Aladdin (1992)                0.833333
Name: 4441, dtype: float64